In [1]:
#pip install NRCLex

In [3]:
# Import required modules
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import requests
from LeXmo import LeXmo
from preprocessing import processing
import pickle
import os

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ellietupin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [18]:
# Import earning calls
ticker = "AAPL"
dir_path = "data/text/"+ticker+"/"
ticker = os.path.basename(os.path.dirname(dir_path))
list_ids = os.listdir(dir_path) 
print(list_ids)
texts = []
for s in list_ids:
    with open(dir_path + s, encoding = 'ISO-8859-1') as f:
        t = f.read()
        texts.append(t)
print('Number of articles', len(texts))
print(ticker)

['4482308', '13884', '280344', '321818', '4441901', '4422230', '2331715', '1364041', '4401478', '61117', '4217359', '100980', '1129431', '1780172', '150291', '264616', '2576865', '745271', '33539', '73697', '42374', '18756', '4142447', '247197', '199800', '4504834', '3991811', '4382943', '3836826', '4463261', '132506', '4362707', '4319666', '4236503', '2856006', '1566012', '86056', '24432', '524451', '1971291', '4119877', '2159213', '4300690', '4068153', '4168271', '3107596', '4192790', '230710', '4279301', '167404', '215492', '4258275', '300433', '952971', '4041266', '5939', '3611256', '3344115', '50846', '184328', '115797', '3968300', '4341792', '4093437', '4014922', '9249']
Number of articles 66
AAPL


In [15]:
# Get dates
data_path = "data/ticker/id_to_date"
with open(data_path,"rb") as file :
    id_to_date = pickle.load(file)
print(len(id_to_date))
print(len(list_ids))

66
66


In [58]:
# Test : create dataframe with earning calls informations
df1 = pd.DataFrame(data={'id': list_ids})
df1['date']=list_ids
df1=df1.replace({'date': id_to_date})
df1['date'] = pd.to_datetime(df1['date'])
df1['quarter'] = df1['date'].dt.to_period('Q')
df1['ticker']= ticker
df1=df1.replace({'ticker': id_to_date})
df1= df1.sort_values(by=['date'], ascending=False)
display(df1)

,id,date,quarter,ticker
25,4504834,2022-04-28,2022Q2,AAPL
0,4482308,2022-01-27,2022Q1,AAPL
29,4463261,2021-10-28,2021Q4,AAPL
4,4441901,2021-07-27,2021Q3,AAPL
5,4422230,2021-04-28,2021Q2,AAPL
...,...,...,...,...
37,24432,2007-01-17,2007Q1,AAPL
21,18756,2006-10-19,2006Q4,AAPL
1,13884,2006-07-19,2006Q3,AAPL
65,9249,2006-04-20,2006Q2,AAPL


In [20]:
# list of all S&P 500 tickers
import pandas as pd
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_tickers = table[0]
df_tickers.to_csv('S&P500-Info.csv')
df_tickers.to_csv("S&P500-Symbols.csv", columns=['Symbol'])
tickers=df_tickers["Symbol"].values
print(tickers)

['MMM' 'AOS' 'ABT' 'ABBV' 'ABMD' 'ACN' 'ATVI' 'ADM' 'ADBE' 'ADP' 'AAP'
 'AES' 'AFL' 'A' 'AIG' 'APD' 'AKAM' 'ALK' 'ALB' 'ARE' 'ALGN' 'ALLE' 'LNT'
 'ALL' 'GOOGL' 'GOOG' 'MO' 'AMZN' 'AMCR' 'AMD' 'AEE' 'AAL' 'AEP' 'AXP'
 'AMT' 'AWK' 'AMP' 'ABC' 'AME' 'AMGN' 'APH' 'ADI' 'ANSS' 'ANTM' 'AON'
 'APA' 'AAPL' 'AMAT' 'APTV' 'ANET' 'AIZ' 'T' 'ATO' 'ADSK' 'AZO' 'AVB'
 'AVY' 'BKR' 'BALL' 'BAC' 'BBWI' 'BAX' 'BDX' 'WRB' 'BRK.B' 'BBY' 'BIO'
 'TECH' 'BIIB' 'BLK' 'BK' 'BA' 'BKNG' 'BWA' 'BXP' 'BSX' 'BMY' 'AVGO' 'BR'
 'BRO' 'BF.B' 'CHRW' 'CDNS' 'CZR' 'CPT' 'CPB' 'COF' 'CAH' 'KMX' 'CCL'
 'CARR' 'CTLT' 'CAT' 'CBOE' 'CBRE' 'CDW' 'CE' 'CNC' 'CNP' 'CDAY' 'CERN'
 'CF' 'CRL' 'SCHW' 'CHTR' 'CVX' 'CMG' 'CB' 'CHD' 'CI' 'CINF' 'CTAS' 'CSCO'
 'C' 'CFG' 'CTXS' 'CLX' 'CME' 'CMS' 'KO' 'CTSH' 'CL' 'CMCSA' 'CMA' 'CAG'
 'COP' 'ED' 'STZ' 'CEG' 'COO' 'CPRT' 'GLW' 'CTVA' 'COST' 'CTRA' 'CCI'
 'CSX' 'CMI' 'CVS' 'DHI' 'DHR' 'DRI' 'DVA' 'DE' 'DAL' 'XRAY' 'DVN' 'DXCM'
 'FANG' 'DLR' 'DFS' 'DISH' 'DIS' 'DG' 'DLTR' 'D' 'DPZ' 'DOV' 'DOW

In [21]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

tickers = df1['ticker'].tolist()
start = datetime.datetime(2019,1,1)
end = datetime.datetime(2021,12,31)

# create empty dataframe
stock_final = pd.DataFrame()
# iterate over each symbol
for i in tickers[:500]:  
    
    # print the symbol which is being downloaded
    print( str(tickers.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, interval = "1m", progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None


0 : AAPL,
1 Failed download:
- AAPL: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
0 : AAPL,
1 Failed download:
- AAPL: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
0 : AAPL,
1 Failed download:
- AAPL: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
0 : AAPL,
1 Failed download:
- AAPL: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
0 : AAPL,
1 Failed download:
- AAPL: 1m data not available for startTime=1546297200 and endTime=1640905200. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
0 : AAPL,
1 Failed download:
- AAPL: 1m data not available f

KeyboardInterrupt: 

In [ ]:
stock_final

In [ ]:
# Preprocessing
n= 10
texts = [processing(x) for x in texts][:n]

In [ ]:
#N=len(texts)

for i in range(n):
    # creating objects
    text=texts[i]
    emo = LeXmo.LeXmo(text)
    emo.pop('text', None)
    emo_array = np.array(list(emo.items()))
    score = emo_array[:,1]
    if i==0:
        con = score
    else :
        con = np.vstack((con, score))

In [ ]:
# Create dataframe
score = np.reshape(con, (n, 10))
column=['anger','anticipation','disgust','fear','joy','negative','positive','sadness','surprise','trust']
df = pd.DataFrame(score, columns = column)

In [ ]:
number_words = np.zeros(n)
for i in range(n):
    number_words[i] = len(texts[i])
df['number_words']=number_words
df['id'] = earning_call_id[:n]
df['date']=earning_call_id[:n]
df=df.replace({'date': date})
df['ticker']= earning_call_id[:n]
df=df.replace({'ticker': id_mapping})
df= df.sort_values(by=['id'], ascending=False)
display(df)

In [ ]:
print(date.get('4509199'))

# A faire pour cette méthode : 
- classer les earning calls par quarter et par stock index
- importer les stock returns et garder seulement ceux des earning calls (mapping permno/ stock index)

# liens utiles

https://betterprogramming.pub/unlocking-emotions-in-text-using-python-6d062b48d71f